In [2]:
# -*- coding: UTF-8 -*-

#1 import

import os, sys
import time
import math

import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt

import akshare as ak
# import talib

#历史数据-实时数据-开发调试-策略回测-模拟交易-实盘交易-运行监控-风险管理

g_locTimeadj = 28800 # 28800 0
g_version = '1.0.1'

print(f"{sys.version} , {sys.executable} , matplotlibrc: {mpl.get_data_path() }")
print(f"{time.strftime('%Y-%m-%d %H:%M:%S',time.localtime(time.time()+g_locTimeadj))} *****import**v{g_version}*** ")


3.10.4 (main, Nov 29 2022, 20:00:25) [GCC 9.4.0] , /home/codespace/.python/current/bin/python3 , matplotlibrc: /home/codespace/.local/lib/python3.10/site-packages/matplotlib/mpl-data
2023-01-08 21:07:21 *****import**v1.0.1*** 


In [6]:
#2 define

timeStamp = int(round(time.time() * 1000) ) #1645371358737 #13位 毫秒 
dateTime = time.strftime('%Y-%m-%d %H:%M:%S',time.localtime(time.time() ) ) #'2017-11-07 16:47:14' %Y-%m-%d %H:%M:%S %Y%m%d
today = time.strftime('%Y%m%d')  # 20220221 #8位

# dirPath = '/home/at/ipynb'
dirPath = '.'
curDir = os.getcwd() #获取当前工作目录路径

eNameFile = 'eName.csv'
eNamePath = os.path.join(dirPath, eNameFile)

eName_df = pd.read_csv(eNamePath)

maTacticFile = 'maTactic.pkl'
maTacticPath = os.path.join(dirPath, maTacticFile)
maTacticdf = pd.read_pickle(maTacticPath)

# maTacticdf = pd.DataFrame( columns = ['SecurityID','rUpDown','Date','winRate','tradeCount','winLim','winCount','perWin','lossLim','lossCount','perLoss','dayLim','OverCount','perOver','cDownLim','cDownCount','percDown'])

# print(f'{timeStamp = } , {dateTime = } , {today = } ')


In [7]:
print(f"{eName_df }")

              En    Cn  Tdx Sina sina_note    dfcf  dfcf_note
0          Index    序号   序号   序号       NaN      序号   df_index
1     SecurityID    代码   代码   代码   bjshsz8      代码         6位
2         Symbol    名称   名称   名称       NaN      名称        NaN
3       Increase   涨幅%  涨幅%  涨跌幅         %     涨跌幅          %
4            New    现价   现价  最新价       NaN     最新价        NaN
..           ...   ...  ...  ...       ...     ...        ...
120   StopReason  停牌原因  NaN  NaN       NaN    停牌原因  拟筹划重大资产重组
121  ReTradeDate  复牌日期  NaN  NaN       NaN  预计复牌时间        NaN
122    TradeDate  交易日期  NaN  NaN       NaN      日期        NaN
123         Open    开盘  NaN  NaN       NaN      开盘        NaN
124        Close    收盘  NaN  NaN       NaN      收盘        New

[125 rows x 7 columns]


In [3]:
#3 akshare spotStock

stock_zh_a_spot_em_df = ak.stock_zh_a_spot_em()
# print(stock_zh_a_spot_em_df)


In [4]:
#4 ST stop

# stock_zh_a_stop_em_df = ak.stock_zh_a_stop_em() #退市

stock_zh_a_st_em_df = ak.stock_zh_a_st_em() #ST股

eName_st_df = eName_df[eName_df.dfcf.isin( stock_zh_a_st_em_df.columns.values.tolist() )]
eName_st_df = eName_df[['dfcf','En' ]].reset_index(drop=True)

st_df = stock_zh_a_st_em_df.copy(deep=True)
st_df.rename(columns=dict(eName_st_df.values.tolist() ), inplace=True)
# print(st_df)

stock_tfp_em_df = ak.stock_tfp_em(date=today)  #停复牌 20220221

eName_stop_df = eName_df[eName_df.dfcf.isin( stock_tfp_em_df.columns.values.tolist() )]
eName_stop_df = eName_df[['dfcf','En' ]].reset_index(drop=True)

stop_df = stock_tfp_em_df.copy(deep=True)
stop_df.rename(columns=dict(eName_stop_df.values.tolist() ), inplace=True)
# print(stop_df)


    Index SecurityID Symbol StopStartDate StopEndDate StopType StopReason  \
0       1     688157   松井股份    2022-03-09  2022-03-22     连续停牌     刊登重要公告   
1       2     301222   浙江恒威    2022-03-09  2022-03-09     盘中停牌     交易异常波动   
2       3     600461   洪城环境    2022-03-09  2022-03-09     停牌一天     刊登重要公告   
3       4     603056   德邦股份    2022-02-28         NaT     连续停牌     刊登重要公告   
4       5     603661   恒林股份    2022-03-01  2022-03-14     连续停牌     刊登重要公告   
5       6     600145  *ST新亿    2022-03-03         NaT     连续停牌     刊登重要公告   
6       7     833874   泰祥股份    2021-11-08         NaT     连续停牌     刊登重要公告   
7       8     832419   路斯股份    2021-06-30  2022-03-10     连续停牌     刊登重要公告   
8       9     832317   观典防务    2021-10-21         NaT     连续停牌     刊登重要公告   
9      10     833994   翰博高新    2021-11-04         NaT     连续停牌      IPO受理   
10     11     600866   星湖科技    2022-03-08  2022-03-21     连续停牌  拟筹划重大资产重组   
11     12     600856  *ST中天    2022-03-08         NaT     连续停牌     交易异常波动   

In [5]:
aSpot = stock_zh_a_spot_em_df.copy(deep=True)
aSpot.drop(aSpot.index, inplace=True)  #表头 空表


#5 block load
xgFile = 'XG.blk'
xgPath = os.path.join(dirPath, xgFile)

# print(f'{xgPath = }')


with open(xgPath, 'r', encoding='utf-8') as f:
    str_data = f.read().splitlines()

stStock = []
stDict = {}
stopStock = []
stopDict = {}

for i in str_data :
    SecurityID = i[-6:]

    if SecurityID in st_df.SecurityID.values.tolist() :
        stStock.append(SecurityID)
        stDict[SecurityID] = st_df.Symbol.loc[st_df["SecurityID"]==SecurityID].values[0]

    if SecurityID in stop_df.SecurityID.values.tolist() :
        stopStock.append(SecurityID)
        stopDict[SecurityID] = stop_df.Symbol.loc[stop_df["SecurityID"]==SecurityID].values[0]

    if len(SecurityID) == 6 :
        # print(f'{SecurityID = } ', len(SecurityID) )
        aSpot = aSpot.append(stock_zh_a_spot_em_df.loc[stock_zh_a_spot_em_df["代码"]==SecurityID] )  #000002 

if stDict :
    print(f'{stDict = }')

if stopDict :
    print(f'{stopDict = }' , '\n')

aSpot = aSpot.reset_index(drop=True)

eName_aSpot = eName_df[eName_df.dfcf.isin( aSpot.columns.values.tolist() )]
eName_aSpot = eName_aSpot[['dfcf','En' ]].reset_index(drop=True)
aSpot.rename(columns=dict(eName_aSpot.values.tolist() ), inplace=True)
print(aSpot)


stDict = {'600090': '*ST济堂', '300256': '*ST星星'}
   Index SecurityID Symbol     New  Increase   Chg        Vol           Amt  \
0   4283     000002  万  科Ａ   16.98     -3.90 -0.69  2299647.0  3.907216e+09   
1   2864     600000   浦发银行    7.90     -2.23 -0.18   460357.0  3.673194e+08   
2   2887     300001    特锐德   20.06     -2.24 -0.46   174768.0  3.511934e+08   
3   3581     688001   华兴源创   29.03     -2.91 -0.87    11564.0  3.371382e+07   
4    384     430047   诺思兰德   14.59      2.75  0.39    10591.0  1.518525e+07   
5   4574     002429   兆驰股份    3.72     -4.62 -0.18   434852.0  1.635745e+08   
6    144     600090  *ST济堂    0.99      5.32  0.05   510810.0  5.026375e+07   
7   3973     300256  *ST星星    2.57     -3.38 -0.09   184927.0  4.757816e+07   
8   4521     688131   皓元医药  157.50     -4.45 -7.34     8776.0  1.414850e+08   

     Amp    High     Low    Open     Pre    VR   Tor      PE     PB  
0  10.07   17.83   16.05   17.77   17.67  1.61  2.37    8.87   0.87  
1   4.58    8.13    7

In [19]:
#股票市场总貌
# stock_sse_summary_df = ak.stock_sse_summary() #上证  股票市场总貌
# print(stock_sse_summary_df)

# stock_szse_summary_df = ak.stock_szse_summary(date="20220222") #深证 股票市场总貌
# print(stock_szse_summary_df)

#股票市场总貌
# stock_sse_deal_daily_df = ak.stock_sse_deal_daily(date="20220221") #error 股票市场总貌
# print(stock_sse_deal_daily_df)

SecurityID = "300418"
#300415 000001 300240 300418 
#个股信息查询
# stock_individual_info_em_df = ak.stock_individual_info_em(symbol="000001") #个股信息查询
# print(stock_individual_info_em_df)

#历史行情数据
# stock_zh_a_hist_df = ak.stock_zh_a_hist(symbol="000001", period="daily", start_date="20170301", end_date='20210907', adjust="qfq")
stock_zh_a_hist_df = ak.stock_zh_a_hist(symbol=SecurityID, period="daily", adjust="qfq") 
#adjust	str	默认不复权的数据; qfq: 前复权后的数据; hfq: 后复权后的数据
# period str period='daily'; choice of {'daily', 'weekly', 'monthly'}
# 日期TradeDate 开盘Open 收盘Close 最高High 最低Low 成交量Vol 成交额Amt 振幅Amp 涨跌幅Increase 涨跌额Chg 换手率Tor 
# print(stock_zh_a_hist_df)





              日期     开盘     收盘     最高     最低     成交量          成交额     振幅  \
0     2015-01-21   5.65   6.89   6.89   5.65     711    2071681.0  26.90   
1     2015-01-22   7.63   7.63   7.63   7.63     187     601205.0   0.00   
2     2015-01-23   8.45   8.45   8.45   8.45     216     762223.0   0.00   
3     2015-01-26   9.35   9.35   9.35   9.35     265    1031115.0   0.00   
4     2015-01-27  10.35  10.35  10.35  10.35    1084    4639092.0   0.00   
...          ...    ...    ...    ...    ...     ...          ...    ...   
1660  2022-03-04  17.35  17.63  17.85  17.28  188560  331572464.0   3.25   
1661  2022-03-07  17.40  17.11  17.40  16.83  188914  323707792.0   3.23   
1662  2022-03-08  17.28  16.60  17.60  16.48  199223  337114528.0   6.55   
1663  2022-03-09  16.65  16.26  16.73  15.50  216201  349908432.0   7.41   
1664  2022-03-10  16.60  16.54  16.90  16.48  171946  286381056.0   2.58   

        涨跌幅   涨跌额   换手率  
0     49.46  2.28  0.10  
1     10.74  0.74  0.03  
2     10.

In [35]:
# kLinedf = 'sz' + SecurityID

kLinedf = stock_zh_a_hist_df.tail(700).copy(deep=True)  #140 280 420 560 700
kLinedf = kLinedf.reset_index(drop=True)

eName_hist = eName_df[eName_df.dfcf.isin( stock_zh_a_hist_df.columns.values.tolist() )]
eName_hist = eName_hist[['dfcf','En' ]].reset_index(drop=True)
kLinedf.rename(columns=dict(eName_hist.values.tolist() ), inplace=True)

# print(kLinedf)


      TradeDate   Open  Close   High    Low     Vol          Amt   Amp  \
0    2019-04-23  14.90  14.88  15.10  14.71  105362  159096516.0  2.59   
1    2019-04-24  14.95  15.34  15.37  14.72  131115  200373501.0  4.37   
2    2019-04-25  15.33  14.57  15.33  14.56  124922  189216501.0  5.02   
3    2019-04-26  14.50  14.74  14.94  14.42  104857  156711000.0  3.57   
4    2019-04-29  15.05  14.88  15.32  14.71  173103  263248915.0  4.14   
..          ...    ...    ...    ...    ...     ...          ...   ...   
695  2022-03-04  17.35  17.63  17.85  17.28  188560  331572464.0  3.25   
696  2022-03-07  17.40  17.11  17.40  16.83  188914  323707792.0  3.23   
697  2022-03-08  17.28  16.60  17.60  16.48  199223  337114528.0  6.55   
698  2022-03-09  16.65  16.26  16.73  15.50  216201  349908432.0  7.41   
699  2022-03-10  16.60  16.54  16.90  16.48  171946  286381056.0  2.58   

     Increase   Chg   Tor  
0       -1.13 -0.17  1.69  
1        3.09  0.46  2.10  
2       -5.02 -0.77  2.00  

In [42]:
for i in list(range(2,20+1) ) + [30,60,90,120,250] :
    kLinedf['Ma'+str(i)] = talib.MA(kLinedf.Close.values, timeperiod=i)

r = 0
sumClose = 0.0
for rindex , row in kLinedf.iterrows():
    r += 1
    sumClose += row['Close']
    maClose = round(sumClose/r, 3)

    for i in list(range(2,20+1) ) + [30,60,90,120,250]:
        Ma = 'Ma'+str(i)
        if math.isnan(row[Ma]):
            kLinedf.at[rindex, Ma] = maClose
        else:
            kLinedf.at[rindex, Ma] = round(row[Ma], 3)          

# print(kLinedf )

In [45]:
for i in range(2,20+1):
    Ma = 'Ma'+str(i)

    cUp = 'cUp'+str(i)
    kLinedf[cUp] = 0
    kLinedf.loc[kLinedf.Close[(kLinedf.Close > kLinedf[Ma]) 
        & (kLinedf.Close.shift() < kLinedf[Ma].shift() )].index, cUp] = 1

    cDown = 'cDown'+str(i)    
    kLinedf[cDown] = 0
    kLinedf.loc[kLinedf.Close[(kLinedf.Close < kLinedf[Ma]) 
        & (kLinedf.Close.shift() > kLinedf[Ma].shift() )].index, cDown] = -1

    # kLinedf['cUp'+str(i)] = kLinedf[['Close',Ma]].apply(lambda row: True if row.Close > row[Ma] 
    #     and row.Close.shift(periods=-1, axis=0) < row[Ma].shift(periods=-1, axis=0) 
    #     else False, axis=1)

for i in [30,60,90,120,250]:
    kLinedf['On'+str(i)] = 0
    kLinedf.loc[kLinedf.Close[(kLinedf.Close > kLinedf.Ma30)].index, 'On'+str(i)] = 1 

# print(kLinedf.columns.values.tolist() )
# print(kLinedf )



In [46]:
winLim = 0.03
lossLim = -0.05
dayLim = 5
cDownLim = -1

for i in range(2,20+1):
    
    cUp = 'cUp'+str(i)

    for j in range(2,20+1):

        tUpDown = 'tU'+str(i) +'D'+str(j)  
        kLinedf[tUpDown] = 0

        # bUpDown = 'bU'+str(i) +'D'+str(j) 
        # kLinedf[bUpDown] = kLinedf.Close

        # sUpDown = 'sU'+str(i) +'D'+str(j) 
        # kLinedf[sUpDown] = kLinedf.Close

        rUpDown = 'rU'+str(i) +'D'+str(j) 
        kLinedf[rUpDown] = 1.0

        cDown = 'cDown'+str(i) 

        kLinedf.loc[(kLinedf[cUp] == 1) & ( (kLinedf['On60'] == 1) | (kLinedf['On90'] == 1) | (kLinedf['On120'] == 1) | (kLinedf['On250'] == 1) ), tUpDown] = 1

        signal = 0
        winRate = 1.0
        posiDay = 0
        posiVol = 0        
        initAmt = 0
        buyPrice = 0.0
        sellPrice = 0.0

        tradeCount = 0
        winCount = 0
        lossCount = 0
        OverCount = 0
        cDownCount = 0
        
        r = 0
        for rindex , row in kLinedf.iterrows():
            r += 1

            signal = row[tUpDown]
            # kLinedf.at[rindex, rUpDown] = winRate

            if posiDay > 0:
                
                kLinedf.at[rindex, rUpDown] = winRate
                
                posiDay += 1
                if row[tUpDown] == 1:
                    signal = 0
                    kLinedf.at[rindex, tUpDown] = signal
                    # continue

                if row["High"] > buyPrice * (1+ winLim):
                    signal = -1
                    posiDay = 0 
                    sellPrice = round(buyPrice * (1+ winLim),2)
                    winRate = round(winRate * sellPrice /buyPrice,3)
                    kLinedf.at[rindex, rUpDown] = winRate

                    winCount += 1

                    # print(rUpDown, rindex, signal, buyPrice, sellPrice, winRate)

                elif row["Low"] < buyPrice * (1+ lossLim):
                    signal = -2
                    posiDay = 0 
                    sellPrice = round(buyPrice * (1+ lossLim),2)
                    winRate = round(winRate * sellPrice /buyPrice,3)
                    kLinedf.at[rindex, rUpDown] = winRate

                    lossCount += 1

                    # print(rUpDown, rindex, signal, buyPrice, sellPrice, winRate) 

                elif row[cDown] == cDownLim:
                    signal = -4
                    posiDay = 0 
                    sellPrice = row["Close"]
                    winRate = round(winRate * sellPrice /buyPrice,3)
                    kLinedf.at[rindex, rUpDown] = winRate

                    cDownCount += 1

                elif posiDay >= dayLim:
                    signal = -3
                    posiDay = 0
                    sellPrice = row["Close"]
                    winRate = round(winRate * sellPrice /buyPrice,3) 
                    kLinedf.at[rindex, rUpDown] = winRate

                    OverCount += 1

                    # print(rUpDown, rindex, signal, buyPrice, sellPrice, winRate)

                kLinedf.at[rindex, tUpDown] = signal

            elif posiDay == 0:
                if row[tUpDown] == 1:
                    signal = 1
                    posiDay = 1
                    buyPrice = row["Close"]

                    tradeCount += 1

            else:
                pass

            kLinedf.at[rindex, rUpDown] = winRate

        perWin = round(winCount/tradeCount*100,1)
        perLoss = round(lossCount/tradeCount*100,1)
        perOver = round(OverCount/tradeCount*100,1)
        percDown = round(cDownCount/tradeCount*100,1)

        # print(f"{SecurityID}:{rUpDown}:{winRate}:交易次数:{tradeCount}, {winLim}止盈:{winCount}次{perWin}%, {lossLim}止损:{lossCount}次{perLoss}%, {dayLim}日超期:{OverCount}次{perOver}%, {cDownLim}死叉:{cDownCount}次{percDown}%  ")

        maTacticdf.loc[maTacticdf.shape[0] ] = [SecurityID, rUpDown, today, winRate, tradeCount, winLim, winCount, perWin, lossLim, lossCount, perLoss, dayLim, OverCount, perOver, cDownLim, cDownCount, percDown]

        # break

# kLinedf.round(2)
# kLinedf.sU2D2.round(2)

# print(kLinedf.sU2D2.values.tolist() )

# print(kLinedf.columns.values.tolist() )

# temp = kLinedf.loc[:, ['Close', 'High', 'Low', 'Ma2', 'Ma3', 'cUp2', 'cUp3', 'On120','On250', 'tU2D2', 'rU2D2', 'tU2D3', 'rU2D3', 'tU3D2', 'rU3D2', 'tU3D3', 'rU3D3' ] ].tail(600)

# print(kLinedf )
# print(kLinedf.loc[:,['Close','Ma2','cUp2','cDown2']].head(30))
# temp = temp.reset_index(drop=True)
# print(temp)


In [47]:
# kLinedf.at[124, 'rU2D2'] = 0.93

# kLinedf.iloc[124,list(kLinedf.columns).index('rU2D2')] = 0.95

# print(kLinedf.at[124, 'rU2D2'])
# print(f"{SecurityID}:{rUpDown}:{winRate}:交易次数:{tradeCount}, {winLim}止盈:{winCount}次{perWin}%, {lossLim}止损:{lossCount}次{perLoss}%, {dayLim}日超期:{OverCount}次{perOver}%, {cDownLim}死叉:{cDownCount}次{percDown}%  ")

# maTacticdf = pd.DataFrame( columns = ['SecurityID','rUpDown','winRate','tradeCount','winLim','winCount','perWin','lossLim','lossCount','perLoss','dayLim','OverCount','perOver','cDownLim','cDownCount','percDown'])

# madf = pd.DataFrame(data=['000001','rU2D2', 1.175, 110, 0.03, 35, 31.8, -0.05, 2, 1.8, 5, 0, 0.0, -1,73,66.4 ],  columns = ['SecurityID','rUpDown','winRate','tradeCount','winLim','winCount','perWin','lossLim','lossCount','perLoss','dayLim','OverCount','perOver','cDownLim','cDownCount','percDown'])

# 000001:rU2D2:1.175:交易次数:110, 0.03止盈:35次31.8%, -0.05止损:2次1.8%, 5日超期:0次0.0%, 死叉:73次66.4% 
# madf.append(pd.Series(['000001','rU2D2', 1.175, 110, 0.03, 35, 31.8, -0.05, 2, 1.8, 5, 0, 0.0, -1,73,66.4 ], index=['SecurityID','rUpDown','winRate','tradeCount','winLim','winCount','perWin','lossLim','lossCount','perLoss','dayLim','OverCount','perOver','cDownLim','cDownCount','percDown'] ), ignore_index=True)

# maTacticdf.loc[maTacticdf.shape[0]] = ['000001','rU2D2', 1.175, 110, 0.03, 35, 31.8, -0.05, 2, 1.8, 5, 0, 0.0, -1,73,66.4 ]
# maTacticdf.loc[maTacticdf.shape[0]] = ['000001','rU2D2', 1.175, 110, 0.03, 35, 31.8, -0.05, 2, 1.8, 5, 0, 0.0, -1,73,66.4 ]


# print(maTacticdf.shape)
# print(maTacticdf)

# maTacticdf.sort_values(by='winRate', ascending=False) # 降序


maTacticdf = maTacticdf.sort_values(['perWin'], ascending=[False]) # 降序 'winRate', 'perWin' , False
print(maTacticdf.describe() )

maTacticdf.to_pickle(maTacticPath)




          winRate        winLim      perWin       lossLim     perLoss  \
count  361.000000  3.610000e+02  361.000000  3.610000e+02  361.000000   
mean     0.912947  3.000000e-02   50.800000 -5.000000e-02   15.963158   
std      0.193399  6.948525e-18    7.709078  1.389705e-17    6.052717   
min      0.575000  3.000000e-02   37.400000 -5.000000e-02    6.900000   
25%      0.745000  3.000000e-02   44.600000 -5.000000e-02   12.500000   
50%      0.945000  3.000000e-02   51.500000 -5.000000e-02   15.200000   
75%      1.076000  3.000000e-02   55.600000 -5.000000e-02   20.600000   
max      1.198000  3.000000e-02   63.300000 -5.000000e-02   26.500000   

          perOver    percDown  
count  361.000000  361.000000  
mean     3.926316   29.305263  
std      2.655759    8.295564  
min      0.000000   18.800000  
25%      2.000000   24.200000  
50%      3.000000   26.200000  
75%      6.200000   32.700000  
max     10.300000   54.500000  


In [23]:
winLim = 0.03
lossLim = 0.05
dayLim = 5

for i in range(2,3+1):
    
    cUp = 'cUp'+str(i)

    for j in range(2,3+1):

        tUpDown = 'tU'+str(i) +'D'+str(j)  
        kLinedf[tUpDown] = 0

        bUpDown = 'bU'+str(i) +'D'+str(j) 
        kLinedf[bUpDown] = kLinedf.Close

        sUpDown = 'sU'+str(i) +'D'+str(j) 
        kLinedf[sUpDown] = kLinedf.Close

        cDown = 'cDown'+str(i) 

        kLinedf.loc[ (kLinedf[cUp] == 1) & ((kLinedf['On120'] == 1 ) | (kLinedf['On250'] == 1 )  )  , tUpDown] = 1

        for k in range(1, dayLim+1):
            kLinedf.loc[ (kLinedf[tUpDown].shift() >= 1) , tUpDown] = kLinedf[tUpDown].shift() +1
            kLinedf.loc[ (kLinedf[tUpDown].shift() <= 0) & (kLinedf[cUp] == 1) & ((kLinedf['On120'] == 1 ) | (kLinedf['On250'] == 1 ) ) , tUpDown] = 1

            kLinedf.loc[ ~ ((kLinedf[tUpDown] == 1 ) | (kLinedf[tUpDown] == -1 ) ) , bUpDown] = kLinedf[bUpDown].shift()


            #止盈 3%  winLim
            kLinedf.loc[ (kLinedf[tUpDown] > 1 ) & ( kLinedf.High >= kLinedf[bUpDown] * (1 +winLim))  , tUpDown] = -1
            kLinedf.loc[ (kLinedf[tUpDown] == -1)  , sUpDown] = kLinedf[bUpDown] * (1 +winLim)

            #止损 5%  lossLim
            kLinedf.loc[ (kLinedf[tUpDown] > 1 ) & ( kLinedf.Low <= kLinedf[bUpDown] * (1 -lossLim))  , tUpDown] = -2
            kLinedf.loc[ (kLinedf[tUpDown] == -2)  , sUpDown] = kLinedf[bUpDown] * (1 -lossLim)
            

            # 平仓日 5  dayLim
            kLinedf.loc[ (kLinedf[tUpDown] == dayLim )  , tUpDown] = -3
            kLinedf.loc[ (kLinedf[tUpDown] == -3)  , sUpDown] = kLinedf.Close

            #信号整理
            kLinedf.loc[ (kLinedf[tUpDown] <= 0) & (kLinedf[cUp] == 1) & ((kLinedf['On120'] == 1 ) | (kLinedf['On250'] == 1 )  )  , tUpDown] = 1

            kLinedf.loc[ (kLinedf[tUpDown].shift() < 1) & (kLinedf[tUpDown] != 1) , tUpDown] = 0

            kLinedf.loc[ (kLinedf[tUpDown] - kLinedf[tUpDown].shift() >= 2)  , tUpDown] = 0

            kLinedf.loc[ (kLinedf[tUpDown].shift() <= 0) & (kLinedf[tUpDown] < 0) , tUpDown] = 0



            kLinedf.loc[ (kLinedf[tUpDown] == 0) , bUpDown] = kLinedf.Close



kLinedf = kLinedf.round(2)


# kLinedf.round(2)
# kLinedf.sU2D2.round(2)

# print(kLinedf.sU2D2.values.tolist() )

print(kLinedf.columns.values.tolist() )

temp = kLinedf.loc[:, ['Close', 'High', 'Low', 'Ma2', 'Ma3', 'cUp2', 'cUp3', 'On120','On250', 'tU2D2', 'bU2D2', 'sU2D2',
    'tU2D3', 'bU2D3', 'sU3D2', 'tU3D2', 'bU3D2', 'tU3D3', 'bU3D3' ] ].tail(600)
    
# print(kLinedf )
# print(kLinedf.loc[:,['Close','Ma2','cUp2','cDown2']].head(30))
# temp = temp.reset_index(drop=True)
print(temp)


['TradeDate', 'Open', 'Close', 'High', 'Low', 'Vol', 'Amt', 'Amp', 'Increase', 'Chg', 'Tor', 'Ma2', 'Ma3', 'Ma4', 'Ma5', 'Ma6', 'Ma7', 'Ma8', 'Ma9', 'Ma10', 'Ma11', 'Ma12', 'Ma13', 'Ma14', 'Ma15', 'Ma16', 'Ma17', 'Ma18', 'Ma19', 'Ma20', 'Ma30', 'Ma60', 'Ma90', 'Ma120', 'Ma250', 'cUp2', 'cDown2', 'cUp3', 'cDown3', 'cUp4', 'cDown4', 'cUp5', 'cDown5', 'cUp6', 'cDown6', 'cUp7', 'cDown7', 'cUp8', 'cDown8', 'cUp9', 'cDown9', 'cUp10', 'cDown10', 'cUp11', 'cDown11', 'cUp12', 'cDown12', 'cUp13', 'cDown13', 'cUp14', 'cDown14', 'cUp15', 'cDown15', 'cUp16', 'cDown16', 'cUp17', 'cDown17', 'cUp18', 'cDown18', 'cUp19', 'cDown19', 'cUp20', 'cDown20', 'On30', 'On60', 'On90', 'On120', 'On250', 'tU2D2', 'bU2D2', 'sU2D2', 'tU2D3', 'bU2D3', 'sU2D3', 'tU3D2', 'bU3D2', 'sU3D2', 'tU3D3', 'bU3D3', 'sU3D3']
     Close   High    Low    Ma2    Ma3  cUp2  cUp3  On120  On250  tU2D2  \
100  12.89  13.34  12.82  13.11  13.15     0     0      0      0    0.0   
101  13.71  14.09  12.91  13.30  13.31     1     1      0

In [ ]:
print(kLinedf.columns.values.tolist() )

print(kLinedf.index[0], type(kLinedf.index[0]) , type(pd.to_numeric(kLinedf.index[0])) )

df = pd.DataFrame({'BoolCol': [1, 2, 3, 3, 4],'attr': [22, 33, 22, 44, 66]}, 
    index=[10,20,30,40,50]) 
print(df) 
a = df[(df.BoolCol==3)&(df.attr==22)].index.tolist() 
b = df[(df.BoolCol==3)&(df.attr==22)].index[0]


print(a,b,type(b))



In [ ]:
#  DF['eee'] = DF[['aaa','bbb']].apply(lambda x: x.aaa * x.bbb, axis=1)

kLinedf['cUp2'] = 0
kLinedf.loc[kLinedf.Close[(kLinedf.Close > kLinedf.Ma2) 
    & (kLinedf.Close.shift() < kLinedf.Ma2.shift() )].index, 'cUp2'] = 1

kLinedf['cDown2'] = 0
kLinedf.loc[kLinedf.Close[(kLinedf.Close < kLinedf.Ma2) 
    & (kLinedf.Close.shift() > kLinedf.Ma2.shift() )].index, 'cDown2'] = -1


print(kLinedf.loc[:,['Close','Ma2','cUp2','cDown2']].head(30))
# print(kLinedf['Close','Ma2','cUp2'])


In [ ]:
maTac = pd.read_pickle('/home/at/ipynb/QL/maTactic_20220317A.pkl')
maTac2 = pd.read_pickle('/home/at/ipynb/QL/maTactic_20220317B.pkl')
maTac= maTac.append(maTac2)
print(maTac.describe())
maTac = maTac.sort_values(['winRate'], ascending=[False])
# print(maTac.describe())

maTac.to_pickle('/home/at/ipynb/QL/maTactic_20220317.pkl')
maTac.to_csv('/home/at/ipynb/QL/maTactic_20220317.csv', index=False)

winMax = maTac.groupby(['SecurityID'], as_index=False)['winRate'].max()
print( winMax )
# print(winMax.values.tolist() )

In [51]:

# winMax = maTac.groupby(['SecurityID'], as_index=False)['winRate'].max()
# print( winMax )
# perWin = maTac.groupby(['SecurityID'], as_index=False)['perWin'].max()
# perLoss = maTac.groupby(['SecurityID'], as_index=False)['perLoss'].min()  #percDown

# sz300390 = maTac.loc[(maTac.SecurityID == '300390') & (maTac.winRate >= 3.9)]
# print( sz300390.describe() )
# print(sz300390)

maTac_winMax = maTac.copy(deep=True)
maTac_winMax.drop(maTac_winMax.index, inplace=True)  # 表头 空表

for rindex, row in winMax.iterrows():
    tmp = maTac.loc[(maTac.SecurityID == row.SecurityID) & (maTac.winRate == row.winRate)]
    # print(tmp.describe())
    # print(tmp[0:])
    maTac_winMax = maTac_winMax.append(tmp[0:1], ignore_index=True)

maTac_winMax = maTac_winMax.sort_values(['winRate'], ascending=[False])
# print(maTac_winMax)



   SecurityID  rUpDown      Date  winRate tradeCount  winLim winCount  perWin  \
15     300390   rU2D15  20220317    3.908         74    0.05       59    79.7   
9      300124   rU3D10  20220317    3.623         58    0.05       41    70.7   
24     300726   rU2D18  20220317    3.407         87    0.03       74    85.1   
31     600702    rU4D9  20220317    2.986         51    0.05       36    70.6   
25     300759    rU3D7  20220317    2.868         57    0.05       42    73.7   
34     601100   rU4D16  20220317    2.797         51    0.05       34    66.7   
33     600862   rU4D20  20220317    2.725         52    0.05       34    65.4   
21     300636   rU5D20  20220317    2.679         42    0.05       32    76.2   
19     300628   rU3D10  20220317    2.625         52    0.05       38    73.1   
16     300482    rU3D9  20220317    2.449         54    0.03       47    87.0   
13     300347    rU8D7  20220317    2.418         40    0.05       26    65.0   
6      300014    rU2D5  2022

In [64]:

sz300390 = maTac.loc[(maTac.SecurityID == '300390') & (maTac.winLim  == 0.05) & (maTac.lossLim  == -0.03) & (maTac.dayLim  == 7)]
sz300390 = sz300390.sort_values(['winRate'], ascending=[False])
sz300390 = sz300390.reset_index(drop=True)
# print( sz300390.describe() )
print(sz300390[0:3])


  SecurityID rUpDown      Date  winRate tradeCount  winLim winCount  perWin  \
0     300390   rU3D9  20220317    2.095         72    0.05       39    54.2   
1     300390  rU3D11  20220317    2.095         72    0.05       39    54.2   
2     300390  rU3D10  20220317    2.095         72    0.05       39    54.2   

   lossLim lossCount  perLoss dayLim OverCount  perOver cDownLim cDownCount  \
0    -0.03        30     41.7      7         0      0.0       -1          3   
1    -0.03        30     41.7      7         0      0.0       -1          3   
2    -0.03        30     41.7      7         0      0.0       -1          3   

   percDown  
0       4.2  
1       4.2  
2       4.2  
